In [1]:
from pathlib import Path
import os
import sys
import json
import copy
import pickle
from hashlib import sha256
import random
import seaborn as sns

In [2]:
import pandas as pd
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import cv2
from tqdm.notebook import tqdm

In [3]:
sys.path.append('..')
from annotation.coco_reader import CocoReader

In [4]:
from modules.helpers import file_functions
from lared.dataset.example import VideoExample, AudioExample, FullExample
from lared.constants.laughter_constants import data_path, cloud_data_path
from lared.data_loading.utils import get_video_caps, AudioFiles

In [5]:
cloud_data_path

'/home/jose/drive/data/lared_laughter'

In [7]:
VideoExample.init_caps(get_video_caps('/media/jose/HDD/data/lared/concat'))
VideoExample.init_camera_annotation(os.path.join(cloud_data_path, 'camera.txt'))
AudioExample.init_dataset(AudioFiles('/media/jose/HDD/data/lared/audio/normalized'))

In [8]:
# load ELAN annotations
laughter_ann_path = os.path.join(cloud_data_path, 'laughter.txt')
# path for output images



laughter_ann = pd.read_csv(laughter_ann_path, sep='\t',names=['name','participant','ini','fin','dur','type'],header=None)
df_laughter_by_participant = laughter_ann.groupby('participant')

In [9]:
dfs_laughter = []
for i in df_laughter_by_participant.groups:
    dfs_laughter.append(df_laughter_by_participant.get_group(i))
participant_ids = df_laughter_by_participant.groups.keys()
participant_ids = [int(id) for id in participant_ids]

In [10]:
sum([len(df) for df in dfs_laughter])

1838

# Laughter examples: loading
load a previously generated set of examples

In [11]:
# read the inputs
# laughter_examples_df = pd.read_csv(os.path.join(cloud_data_path, 'laughter_data', 'laughter_examples', 'examples.csv'), index_col=0)
# laughter_examples = {ex[1].hash: FullExample(**ex[1].to_dict()) for ex in laughter_examples_df.iterrows()}
# speech_examples_df = pd.read_csv(os.path.join(cloud_data_path, 'laughter_data', 'speech_examples', 'examples.csv'), index_col=0)
# speech_examples = {ex[1].hash: FullExample(**ex[1].to_dict()) for ex in speech_examples_df.iterrows()}

laughter_bbs = CocoReader(os.path.join(cloud_data_path, 'laughter_data', 'laughter_examples', 'cvat.json'))
speech_bbs = CocoReader(os.path.join(cloud_data_path, 'laughter_data', 'speech_examples', 'cvat.json'))

In [12]:
examples = pd.read_csv('../annotation/analysis/annotation_results.csv')
examples = examples.groupby('hash').agg({
    'hash': 'first',
    'cam': 'first',
    'person': 'first',
    'ini_time': 'first',
    'end_time': 'first'
}).rename(columns={'person':'pid'})

In [13]:
examples.head()

,hash,cam,pid,ini_time,end_time
hash,,,,,
002043005fa746f076c845dc38dd1bd97327bde09e17fcb953102c7ff0b277ac,002043005fa746f076c845dc38dd1bd97327bde09e17fc...,2,9,7963.610000,7965.490000
006f74addfc99845bf6c9f80d13d52ccc189341031525530762bb83dd8b713af,006f74addfc99845bf6c9f80d13d52ccc1893410315255...,4,30,8289.657000,8292.107000
0070b1ba6e8de828aea5c6d3b9d1c3662959fb5f032a679cac78e5ca98fcb1ff,0070b1ba6e8de828aea5c6d3b9d1c3662959fb5f032a67...,1,22,1996.955000,2000.122000
00bcb7cfe796c64cf5fa4195248553f4a5937897650cee2e2bc2d80fba782667,00bcb7cfe796c64cf5fa4195248553f4a5937897650cee...,3,22,3700.565000,3701.599000
0160da1890b3fc4a923586e460bb781ada3bc56ea9908aea299ce3828a629c42,0160da1890b3fc4a923586e460bb781ada3bc56ea9908a...,1,4,3526.003181,3532.905657


In [14]:
examples = {ex[1].hash: FullExample(**ex[1].to_dict()) for ex in examples.iterrows()}

In [15]:
len(examples)

510

In [16]:
# output paths
out_path = os.path.join(cloud_data_path, 'laughter_data', 'ml_datasets', 'tight')
audio_path = os.path.join(out_path, 'audio')
video_path = os.path.join(out_path, 'video')
av_path = os.path.join(out_path, 'av')
if not os.path.exists(audio_path): os.makedirs(audio_path)
if not os.path.exists(video_path): os.makedirs(video_path)
if not os.path.exists(av_path): os.makedirs(av_path)

In [18]:
# write back the bbs
for hash, ex in tqdm(examples.items()):
    if hash not in laughter_bbs and hash not in speech_bbs:
        continue
    ex.rect = laughter_bbs[hash] if hash in laughter_bbs else speech_bbs[hash]
    if not ex.rect:
        continue
    ex.write_audio(audio_path, fname=hash, padded=False)
    # ex.write_audio_video(audio_path, './audio.png', aiv_path)
    ex.write_video(video_path, fname=hash, padded=False, enlarge_perc=0)
    ex.write_audiovisual(video_path, audio_path, av_path, fname=hash)

  0%|          | 0/510 [00:00<?, ?it/s]

OpenCV: FFMPEG: tag 0x34363248/'H264' is not supported with codec id 27 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x31637661/'avc1'
OpenCV: FFMPEG: tag 0x34363248/'H264' is not supported with codec id 27 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x31637661/'avc1'
OpenCV: FFMPEG: tag 0x34363248/'H264' is not supported with codec id 27 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x31637661/'avc1'
OpenCV: FFMPEG: tag 0x34363248/'H264' is not supported with codec id 27 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x31637661/'avc1'
OpenCV: FFMPEG: tag 0x34363248/'H264' is not supported with codec id 27 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x31637661/'avc1'
OpenCV: FFMPEG: tag 0x34363248/'H264' is not supported with codec id 27 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x31637661/'avc1'
Open

In [ ]:
examples_dicts = [ex.to_dict() for ex in laughter_examples.values()]
pd.DataFrame(examples_dicts).to_csv(os.path.join(cloud_data_path, 'laughter_data/laughter_examples/examples.csv'))